Here is a basic example of training two models to work on a multi-modal dataset, one using image data and another not. Data in our pipeline will be directed to the appropriate model according to whether image data is available.

This is one of two approaches you could take to a multi-modal workflow. Another is to use a single model trained on multiple data formats which can handle the presence of one format and the absence of another. 

After training your model, you can encode it in .pkl and upload it to S3. You'll be able to pass that 


In [2]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
np.random.seed(123)

In [11]:
for d, di, file in os.walk("test"):
    print(f"{d}, {di}, {file}")

test, ['tabular', 'img'], ['log_python.txt']
test/tabular, [], ['test.csv']
test/img, [], ['0a5a7452a.jpg', '0b41e22a9.jpg', '0ad970d41.jpg', '0a8e773aa.jpg', '0a00680eb.jpg', '0a035caf2.jpg', '0b04d7007.jpg', '0b10de1b6.jpg', '0a372b0d4.jpg', '0ad8ed80c.jpg', '0b7b6d9c1.jpg', '000aa306a.jpg', '0a9cbab15.jpg', '0a3d2b273.jpg', '0b5242b0d.jpg', '0a6d1c439.jpg', '00c8a5254.jpg', '0bbbc724a.jpg']


In [3]:
def process_data(df:pd.DataFrame, split:str) -> pd.DataFrame:
    """
    split (str): train or test 
    """
    
    df2 = df.copy()
    df2.fillna({"Name":"Unnamed", "Description":"No description"}, inplace=True)

    img_dir = f"{os.path.abspath('data')}/{split}_images/"
    images = [(f"{img_dir}/{file}", file[:-6]) for file in os.listdir(img_dir) if "-1.jpg" in file]
    images = pd.DataFrame(images, columns=['Image', 'PetID']).set_index("PetID")

    df2 = df2.merge(images, how="left", left_on="PetID", right_index=True)

    return df2

In [ ]:
# Read Data
data = pd.read_csv("data/train/train.csv")
data = process_data(data, "train")

In [5]:
data['AdoptionSpeed'].value_counts()

4    4197
2    4037
3    3259
1    3090
0     410
Name: AdoptionSpeed, dtype: int64

1. The testing data provided does not have AdoptionSpeed label -- probably because the testing data is what the competition holders were using to grade performance. We will take both our training and testing samples from train.csv. 

2. To simplify task and improve performance, we will group Adoption Speed into two categories: Fast (3-4) and Slow (0-1). We'll drop the middle ranking of 2

In [6]:
data = data[data['AdoptionSpeed'] != 2]
data['AdoptionSpeed'] = np.where(data['AdoptionSpeed'] > 1, 1, 0)

In [7]:
# Filter by images 
has_images = data[~data['Image'].isna()]

In [8]:
# Training and testing samples 
image_sample = has_images.sample(1000) 

train_sample = image_sample.sample(800)
test_sample = image_sample[~image_sample.index.isin(train_sample.index)]

### Train with Images 

In [ ]:
from autogluon.multimodal import MultiModalPredictor
predictor = MultiModalPredictor(label="AdoptionSpeed", 
                                presets='medium_quality')
predictor.fit(
    train_data=train_sample,
    time_limit=600, # seconds
)

In [17]:
# Crashed -- since training progressed enough to generate checkpoint, resume from there
predictor = MultiModalPredictor.load("/home/ec2-user/AutogluonModels/ag-20230418_201348", 
                                    resume=True)

Resume training from checkpoint: '/home/ec2-user/AutogluonModels/ag-20230418_201348/last.ckpt'


In [18]:
predictor.fit(train_sample)

Global seed set to 123
AutoMM starts to create your model. ✨

- Model will be saved to "/home/ec2-user/AutogluonModels/ag-20230418_201348".

- Validation metric is "roc_auc".

- To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /home/ec2-user/AutogluonModels/ag-20230418_201348
    ```

Enjoy your coffee, and let AutoMM do the job ☕☕☕ Learn more at https://auto.gluon.ai

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /home/ec2-user/AutogluonModels/ag-20230418_201348/last.ckpt

  | Name              | Type                | Params
----------------------------------------------------------
0 | model             | MultimodalFusionMLP | 20.3 M
1 | validation_metric | AUROC               | 0     
2 | loss_func         | CrossEntropyLoss    

Epoch 2:  33%|████▎        | 40/120 [01:37<03:14,  2.43s/it, loss=0.883, v_num=]
Validation: 0it [00:00, ?it/s]
Epoch 2:  50%|██████▌      | 60/120 [01:51<01:51,  1.86s/it, loss=0.883, v_num=]
                                                                                

Epoch 2, global step 12: 'val_roc_auc' reached 0.83602 (best 0.83602), saving model to '/home/ec2-user/AutogluonModels/ag-20230418_201348/epoch=2-step=12.ckpt' as top 3


Epoch 2:  83%|███████████▋  | 100/120 [03:21<00:40,  2.02s/it, loss=0.9, v_num=]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████████| 120/120 [03:36<00:00,  1.80s/it, loss=0.9, v_num=]
                                                                                

Epoch 2, global step 15: 'val_roc_auc' reached 0.82840 (best 0.83602), saving model to '/home/ec2-user/AutogluonModels/ag-20230418_201348/epoch=2-step=15.ckpt' as top 3


...

Epoch 7, global step 37: 'val_roc_auc' was not in top 3


Epoch 7:  50%|██████▌      | 60/120 [01:53<01:53,  1.89s/it, loss=0.587, v_num=]


Start to fuse 3 checkpoints via the greedy soup algorithm.


Predicting DataLoader 0: 100%|████████████████████| 5/5 [00:22<00:00,  4.52s/it]


AutoMM has created your model 🎉🎉🎉

- To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/home/ec2-user/AutogluonModels/ag-20230418_201348")
    ```

- You can open a terminal and launch Tensorboard to visualize the training log:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /home/ec2-user/AutogluonModels/ag-20230418_201348
    ```

- If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub: https://github.com/autogluon/autogluon




In [19]:
# On completion, load in again without "resume" parameter
from autogluon.multimodal import MultiModalPredictor
predictor = MultiModalPredictor.load("/home/ec2-user/AutogluonModels/ag-20230418_201348")


Load pretrained checkpoint: /home/ec2-user/AutogluonModels/ag-20230418_201348/model.ckpt


In [21]:
predictor.save("models/mm-binclass-model") # saving final model 

Predicting DataLoader 0: 100%|████████████████████| 7/7 [00:26<00:00,  3.76s/it]


{'roc_auc': 0.5962393162393163}

### Train without Images 
Train non-image model on same data with image column dropped 

In [17]:
train_no_images = train_sample.drop("Image", axis=1)

In [18]:
predictor = MultiModalPredictor(label="AdoptionSpeed", 
                                presets='medium_quality')
predictor.fit(
    train_data=train_no_images,
    time_limit=600, # seconds
)

Global seed set to 123
No path specified. Models will be saved in: "AutogluonModels/ag-20230420_202741/"
AutoMM starts to create your model. ✨

- Model will be saved to "/home/ec2-user/AutogluonModels/ag-20230420_202741".

- Validation metric is "roc_auc".

- To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /home/ec2-user/AutogluonModels/ag-20230420_202741
    ```

Enjoy your coffee, and let AutoMM do the job ☕☕☕ Learn more at https://auto.gluon.ai

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name              | Type                | Params
----------------------------------------------------------
0 | model             | MultimodalFusionMLP | 13.7 M
1 | validation_metric | AUROC               | 0     
2 | loss_func         | CrossEntropyLoss    | 0     
--------------

Epoch 0:  33%|████▋         | 40/120 [01:16<02:33,  1.92s/it, loss=1.18, v_num=]
Validation: 0it [00:00, ?it/s]
Epoch 0:  50%|███████       | 60/120 [01:27<01:27,  1.47s/it, loss=1.18, v_num=]
                                                                                

Epoch 0, global step 2: 'val_roc_auc' reached 0.52407 (best 0.52407), saving model to '/home/ec2-user/AutogluonModels/ag-20230420_202741/epoch=0-step=2.ckpt' as top 3


Epoch 0:  83%|██████████  | 100/120 [02:37<00:31,  1.57s/it, loss=0.925, v_num=]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|████████████| 120/120 [02:48<00:00,  1.40s/it, loss=0.925, v_num=]
                                                                                

Epoch 0, global step 5: 'val_roc_auc' reached 0.56551 (best 0.56551), saving model to '/home/ec2-user/AutogluonModels/ag-20230420_202741/epoch=0-step=5.ckpt' as top 3


...

Epoch 3, global step 18: 'val_roc_auc' reached 0.55863 (best 0.56551), saving model to '/home/ec2-user/AutogluonModels/ag-20230420_202741/epoch=3-step=18.ckpt' as top 3


Epoch 3:  74%|█████████▋   | 89/120 [01:48<00:37,  1.22s/it, loss=0.835, v_num=]

Start to fuse 3 checkpoints via the greedy soup algorithm.



Predicting DataLoader 0: 100%|████████████████████| 5/5 [00:16<00:00,  3.38s/it]


AutoMM has created your model 🎉🎉🎉

- To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/home/ec2-user/AutogluonModels/ag-20230420_202741")
    ```

- You can open a terminal and launch Tensorboard to visualize the training log:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /home/ec2-user/AutogluonModels/ag-20230420_202741
    ```

- If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub: https://github.com/autogluon/autogluon




In [20]:
predictor.save("models/nim-binclass-model")